In [ ]:
#updated SQL query to identify bp readings following care home admission 
CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_bp_post_admission` as (
#selects all bp readings for the care home cohort in those with a diagnosis of hypertension taken after admission date
with bp_primary_care as (
SELECT 
a.person_id, 
b.ctv3code, 
b.ctv3text,
CAST(b.numericvalue as bignumeric) bp_value,
b.dateevent,
a.first_episodestartdate, 
FROM `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_hypertension_combined` a
INNER JOIN `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode` b
ON a.person_id = b.person_id
WHERE ctv3code in(SELECT  CTV3_CONCEPTID FROM `yhcr-prd-bradfor-bia-core.CB_2172.bloodpressure_codes_snomed_codes_rough`)
    and (a.hypertension = TRUE) and (dateevent > first_episodestartdate)),
# filter out those with a value less than 10 as these will be not be true values, duplicate_count, 
bp_primary_care_filtered as (
select
*,
row_number() over (partition by bp_primary_care.bp_value, dateevent, person_id) as dup_count
 from bp_primary_care
 where bp_value > 10),
# remove duplicates
bp_readings_filtered_no_duplicates as (
select
* 
from bp_primary_care_filtered
where dup_count=1),

diastolic as (
    SELECT
  *, 
  bp_value as diastolic_value,
  FROM bp_readings_filtered_no_duplicates
  WHERE ctv3text like "%iastolic%"),

  systolic as (
    SELECT 
    *,
    bp_value as systolic_value
    FROM bp_readings_filtered_no_duplicates
    WHERE ctv3text like "%ystolic%"),

reading_join as (
  SELECT 
  diastolic.person_id,
  systolic_value,
  diastolic_value,
  diastolic.dateevent as datetimeevent_diastolic,
  systolic.dateevent as datetimeevent_systolic,
   diastolic.first_episodestartdate,
  extract(date from systolic.dateevent) as date_systolic,
  FROM diastolic
  FULL JOIN systolic 
  ON 
  diastolic.person_id = systolic.person_id AND diastolic.dateevent = systolic.dateevent
  #here i have filtered out reading which, were they a true measurement of a person's blood pressure, would result in urgnet hospital admission or indicate someone who is dying. these values which chosen empirically based on my own clinical knowledge. 
  WHERE datetime_diff(diastolic.dateevent,diastolic.first_episodestartdate,day) < 366 AND systolic_value >70 AND systolic_value <250 AND diastolic_value >49 AND diastolic_value <200),

  filter_daily as (
#here i have selected the first blood pressure recording following admission. 
  SELECT
  *,
  rank() OVER (PARTITION BY person_id ORDER BY datetimeevent_systolic) as bp_seq,
  row_number() over (partition by person_id, date_systolic order by systolic_value) as systolic_daily
  from 
  reading_join rj)

select
* 
from filter_daily
where systolic_daily = 1 and bp_seq = 1
order by person_id)
